In [1]:
import xarray as xr
import numpy as np

In [2]:
# random fake dataset
da = xr.DataArray(np.random.randn(2, 3, 2), dims=("x", "y",'t'), coords={"x": [10, 20], 'y':[33,44,55], 't':[7,8]})
da=da.rename('elev')
ds=da.to_dataset()

In [3]:
ds

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 0.6857 -1.746 -1.435 ... 1.015 1.278 -0.8281

In [4]:
# function to apply
def fn(x,y,elev,z):
    misc_arr = np.array([x,y/2,elev*z])
    return misc_arr

In [5]:
# using groupby and apply
ds.groupby('t').apply(fn, args=(ds.x.values, ds.y.values, ds.elev.values, 7))

TypeError: fn() takes 4 positional arguments but 5 were given

In [6]:
# using groupby and apply with args
gb=ds.groupby('t')
gb.apply(fn, args=(gb.x.values, gb.y.values, gb.elev.values))

AttributeError: 'DatasetGroupBy' object has no attribute 'x'

In [10]:
# create a groupby wrapper to deal with the args issues
def fn_wrapper(gb):
    x=gb.x.values
    y=gb.y.values
    elev=gb.elev.values
    z = 12
    
    array = fn(x,y,elev,z)
    print(array)
    print(gb)
# other things I've tried and their outcomes
#     return gb # successfully evals, new arrays not added to dataset
#     print(gb.assign(new_var=('t',array))) # ValueError: dimension 't' already exists as a scalar variable
    gb['new_var'] = array # AttributeError: 'numpy.ndarray' object has no attribute 'dims'
    return array


In [8]:
# I can successfully compute array, but cannot get it back into the groupby dataarray
ds.groupby('t').apply(fn_wrapper)

[array([10, 20]) array([16.5, 22. , 27.5])
 array([[  8.2286161 , -17.21547738,   2.79377779],
       [-20.00887911,   5.97879762,  15.33385259]])]
<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
    t        int64 7
Data variables:
    elev     (x, y) float64 0.6857 -1.435 0.2328 -1.667 0.4982 1.278


AttributeError: 'numpy.ndarray' object has no attribute 'dims'

In [11]:
# try another approach
# first add the new variable to the dataset before running the function
ds=ds.assign(new_var=('t',[14,15]))
ds

<xarray.Dataset>
Dimensions:  (t: 2, x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
  * t        (t) int64 7 8
Data variables:
    elev     (x, y, t) float64 0.6857 -1.746 -1.435 ... 1.015 1.278 -0.8281
    new_var  (t) int64 14 15

In [12]:
# uncomment ds['new_var'] = array line in function
ds.groupby('t').apply(fn_wrapper)

[array([10, 20]) array([16.5, 22. , 27.5])
 array([[  8.2286161 , -17.21547738,   2.79377779],
       [-20.00887911,   5.97879762,  15.33385259]])]
<xarray.Dataset>
Dimensions:  (x: 2, y: 3)
Coordinates:
  * x        (x) int64 10 20
  * y        (y) int64 33 44 55
    t        int64 7
Data variables:
    elev     (x, y) float64 0.6857 -1.435 0.2328 -1.667 0.4982 1.278
    new_var  int64 14


MergeError: unable to determine if these variables should be coordinates or not in the merged result: {'new_var'}